In [ ]:
#  This is filename:  Detrend_filters.ipynb

#  The purpose of this notebook is to compute
#  trend and cyclical components of time series
#  using the Hodrick-Prescott filter or other filters
#  such as taking first differences, Baxter-King,
#  Christiano-Fitzgerald

#   Here are some available detrending filters
#
#  Christiano Fitzgerald asymmetric, random walk filter.
#  cycle, trend = statsmodels.tsa.filters.cf_filter.cffilter()
#
#  Baxter-King filter
#  cyclical = statsmodels.tsa.filters.bk_filter.bkfilter()
#
#  Hodrick-Prescott filter
#  cycle, trend = statsmodels.tsa.filters.bk_filter.hpfilter()
#
#  Decompose a time series using moving averages
#  decomposeresult = statsmodels.tsa.seasonal.seasonal_decompose()

#  Below I use the Hodrick-Prescott Filter

#  See: (1) R.J. Hodrick and E. C. Prescott. 1980. 
#  “Postwar U.S. Business Cycles: An Empricial 
#  Investigation.” Carnegie Mellon University 
#  discussion paper no. 451.  (2) M.O. Ravn and 
#  H. Uhlig. 2002. “Notes On Adjusting the Hodrick-Prescott
#  Filter for the Frequency of Observations.” The Review 
#  of Economics and Statistics, 84(2), 371-80.

#  If you are taking inputs into this notebook from 
#  simulation results, the the inputs are the matrices 
#  Resp_mat and Resp_mat_named. 
#  You can also upload time series to be filtered. 

#  Output matrices are Mat_trend, Mat_trend_named,
#  Mat_cycle, Mat_cycle_named


import numpy as np
import numpy.matlib as npm
from numpy import hstack, vstack, zeros, dot, eye, kron

import scipy as sp
from scipy import linalg as la
from scipy.linalg import null_space
import scipy.optimize as opt

import matplotlib.pyplot as plt
import pylab

import pandas as pd

import statsmodels.api as sm

from IPython.display import display, HTML

np.set_printoptions(precision=3, suppress=False)
pd.set_option('display.float_format', '{:.4e}'.format)


##  USER INPUT:  Date format


##  USER INPUT:  Choose smoothing parameter Lambda
#  HP_Lambda = 100 for annual
#  HP_Lambda = 1600 for quarterly
#  HP_Lambda = 14400 for monthly
#  Or some other value

HP_Lambda = 1600


##  USER INPUT:  Series to be filtered
##  with start date, end dates and frequency
##  Below calls from simulation results

Input_mat = np.matrix(Response)
Input_mat_named = np.matrix(Response_named[:,0])

Resp_mat = np.matrix(Response)
Resp_mat_named = np.matrix(Response_named)



# Initializing and computing trend and cycle matrices

Mat_trend = np.zeros(Input_mat.shape[0]*Input_mat.shape[1]).reshape(Input_mat.shape[0],Input_mat.shape[1])
Mat_cycle = np.zeros(Input_mat.shape[0]*Input_mat.shape[1]).reshape(Input_mat.shape[0],Input_mat.shape[1])

variable_counter = 0

for variable_counter in range(0,Input_mat.shape[0]):
    cycle, trend = sm.tsa.filters.hpfilter(Input_mat[variable_counter,:], HP_Lambda)
    Mat_trend[variable_counter,:] = trend
    Mat_cycle[variable_counter,:] = cycle

Mat_trend = np.matrix(Mat_trend)
Mat_cycle =  np.matrix(Mat_cycle)



print('  ')
print('Trend and cycle components have been computed.')
print('The matrices Mat_trend, Mat_trend_named,')
print('Mat_cycle, Mat_cycle_named can be downloaded')
print('to excel or used to make plots.')
print('  ')
print('Detrending Filter Used:  Hodrick-Prescott')
print('  ')
print('  ')
print('Simulated Series, Trend Component, Cyclical Comoponent')
print('x-axis: time periods,  y-axis:  %-deviation from steady state')
print('  ')



variable_counter = 0

for variable_counter in range(0,Input_mat.shape[0]):
    
    fig = plt.figure(figsize = (21, 4.8))
    
    x = range(0,(Input_mat.shape[1]))
    var_x = np.array(x).reshape(Input_mat.shape[1],1)

    #  Plotting the time series
    
    plt.subplot(1,3,1)
    
    y=Input_mat[variable_counter,:]
    var_y = y.reshape(Input_mat.shape[1],1)
    #print('  ')
    #print(variable_counter)
    #print('  ')
    #print(var_y)
    #print('  ')

    plt.plot(var_x, var_y, color='blue', linestyle='solid', linewidth = 0.5, marker='o', markerfacecolor='blue', markersize=0.5, label=Input_mat_named[variable_counter,0]+np.str(' simulated series'))  
    pylab.legend(loc='best')
    #pylab.show()
    #plt.show()     

    #  Plotting the trend
 
    plt.subplot(1,3,2)
           
    y=Mat_trend[variable_counter,:]
    var_y = y.reshape(Mat_trend.shape[1],1)
    #print('  ')
    #print(variable_counter)
    #print('  ')
    #print(var_y)
    #print('  ')

    plt.plot(var_x, var_y, color='green', linestyle='solid', linewidth = 0.5, marker='o', markerfacecolor='green', markersize=0.5, label=Input_mat_named[variable_counter,0]+np.str(' TREND')) 
    pylab.legend(loc='best')
    #pylab.show()
    #plt.show() 

    #  Plotting the cyclical component
    
    plt.subplot(1,3,3)
    
    y=Mat_cycle[variable_counter,:]
    var_y = y.reshape(Mat_cycle.shape[1],1)
    #print('  ')
    #print(variable_counter)
    #print('  ')
    #print(var_y)
    #print('  ')

    plt.plot(var_x, var_y, color='black', linestyle='solid', linewidth = 0.5, marker='o', markerfacecolor='black', markersize=0.5, label=Input_mat_named[variable_counter,0]+np.str(' CYCLE'))  
    pylab.legend(loc='best')
    #pylab.show()
    #plt.show() 
    
    pylab.show()
    plt.show() 

    
print('  ')
print('  ')
print('Shocks used in simulation')
print('  ')
    
EPS_counter = 0

if given_EPS.shape[0] == 1:
    
    var_EPS = (given_EPS[EPS_counter,sim_LENGTH_discard:sim_LENGTH_total]).reshape((sim_LENGTH_total - sim_LENGTH_discard),1)
    plt.plot(var_x, var_EPS, color='grey', linestyle='solid', linewidth = 0.5, marker='o', markerfacecolor='grey', markersize=0.5, label='given_EPS - '+str(EPS_counter+1))
    pylab.legend(loc='best')
    pylab.show()
    plt.show()
    
    label_stack = 'given_EPS - '+str(1)
    
else:     

    label_last = 'given_EPS - '+str(1)

    for EPS_counter in range(0,given_EPS.shape[0]):
        var_EPS = (given_EPS[EPS_counter,sim_LENGTH_discard:sim_LENGTH_total]).reshape((sim_LENGTH_total - sim_LENGTH_discard),1)
        plt.plot(var_x, var_EPS, color='grey', linestyle='solid', linewidth = 0.5, marker='o', markerfacecolor='grey', markersize=0.5, label='given_EPS - '+str(EPS_counter+1))
        pylab.legend(loc='best')
        pylab.show()
        plt.show()
        label='given_EPS - '+str(EPS_counter+1)
        label_stack= np.vstack((label_last,label))
        label_last = label

    
#  For convenience

Mat_trend_named = np.hstack((Varnames, Mat_trend))
Mat_cycle_named = np.hstack((Varnames, Mat_cycle))

print('   ')
#print('   ')
#print('Mat_trend_named = ')
#print(Mat_trend_named)
#print('   ')
#print('Mat_cycle_named = ')
#print(Mat_cycle_named)
print('   ')


label_stack = np.array(label_stack).reshape((given_EPS.shape[0]),1)
given_EPS_named = np.hstack((label_stack, given_EPS))


#print('given_EPS_named = ')
#print(given_EPS_named)
#print('   ')


#  To download results into an excel spreadsheet

#writer = pd.ExcelWriter(r'C:\Users\MacTj\aa C0103 WIP\filter_xx_dddddd.xlsx', engine='xlsxwriter')

#Mat_trend_named_df = pd.DataFrame(Mat_trend_named)
#Mat_trend_named_df.to_excel(writer, sheet_name='Mat_trend_named')

#Mat_cycle_named_df = pd.DataFrame(Mat_cycle_named)
#Mat_cycle_named_df.to_excel(writer, sheet_name='Mat_cycle_named')

#writer.save()

#print('Trend and cycle matrices were download to an excel spreadsheet.')
